In [2]:
# $example on$
from pyspark.ml.clustering import LDA
# $example off$
from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("LDAExample") \
        .getOrCreate()

In [10]:
    # $example on$
   # Loads data.
dataset = spark.read.format("libsvm").load("G:/Users/Gabriel/Documents/Education/UoB/GitHubDesktop/DST-Assessment-05/Gabriel Grant/data/sample_lda_libsvm_data.txt")

In [11]:
# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)
# $example off$

spark.stop()

The lower bound on the log likelihood of the entire corpus: -813.879271217884
The upper bound on perplexity: 3.1303048892995537
The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[9, 1, 0]  |[0.09898930760347799, 0.09867275979504042, 0.09806114548986557]|
|1    |[5, 3, 4]  |[0.10691102825055714, 0.10454878605724639, 0.0978255491598731] |
|2    |[9, 5, 7]  |[0.10044863398767978, 0.09872750913906204, 0.09764080968798168]|
|3    |[6, 7, 1]  |[0.10573753744328822, 0.09994253910135988, 0.09511948150860375]|
|4    |[9, 4, 5]  |[0.11499398386791733, 0.1019941041405907, 0.09682165469365621] |
|5    |[10, 6, 3] |[0.2659996672984301, 0.16767612470926174, 0.10698914877006746] |
|6    |[7, 10, 1] |[0.1030389578662118, 0.09649740080041361, 0.095

In [16]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
from pyspark import SparkContext

In [17]:
sc = SparkContext(appName="LatentDirichletAllocationExample")

In [19]:
data = sc.textFile("data/sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

In [23]:
help(corpus)

Help on PipelinedRDD in module pyspark.rdd object:

class PipelinedRDD(RDD)
 |  PipelinedRDD(prev, func, preservesPartitioning=False, isFromBarrier=False)
 |  
 |  Examples
 |  --------
 |  Pipelined maps:
 |  
 |  >>> rdd = sc.parallelize([1, 2, 3, 4])
 |  >>> rdd.map(lambda x: 2 * x).cache().map(lambda x: 2 * x).collect()
 |  [4, 8, 12, 16]
 |  >>> rdd.map(lambda x: 2 * x).map(lambda x: 2 * x).collect()
 |  [4, 8, 12, 16]
 |  
 |  Pipelined reduces:
 |  
 |  >>> from operator import add
 |  >>> rdd.map(lambda x: 2 * x).reduce(add)
 |  20
 |  >>> rdd.flatMap(lambda x: [x, x]).reduce(add)
 |  20
 |  
 |  Method resolution order:
 |      PipelinedRDD
 |      RDD
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, prev, func, preservesPartitioning=False, isFromBarrier=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  getNumPartitions(self)
 |      Returns the number of partitions in RDD
 |      
 |      Examples
 |      ----

In [27]:
dfs = spark.read.load("NCSC Reports.csv",format='csv', header = "true")

Py4JJavaError: An error occurred while calling o206.load.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(Unknown Source)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(Unknown Source)
java.lang.reflect.Constructor.newInstance(Unknown Source)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Unknown Source)

The currently active SparkContext was created at:

org.apache